In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 5.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=0234ab0a944cf50b48fdebb79ae1b92979a3151cea0941de417adda9414a9f52
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=9ee5d43be87603be58891203a900accf17e36f0318a49df7bea307660958ed49
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [24]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
     keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values=[3,5]),
        activation='relu',
    ),
     keras.layers.Flatten(),
     keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'   
     ),
     keras.layers.Dense(10, activation='softmax')#output layer
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [28]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [26]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name='Mnist Fashion'
                          )

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)


Search: Running Trial #4

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |128               |64                
conv_1_kernel     |3                 |5                 
conv_2_filter     |64                |32                
conv_2_kernel     |5                 |3                 
dense_1_units     |80                |96                
learning_rate     |0.001             |0.001             

Epoch 1/3
1510/1688 [=========================>....] - ETA: 1:07 - loss: 0.3975 - accuracy: 0.8557

In [31]:
model=tuner_search.get_best_models(num_models=1)[0]

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        18464     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                1486944   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 1,508,042
Trainable params: 1,508,042
Non-trainable params: 0
_________________________________________________________________
